In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base').eval()

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').eval()

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlm': xlm}

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [29]:
with open('stockholm/mlm/literal_mask_acapo.txt', 'r') as fp:
    text = fp.read().split('\n')

In [32]:
text

["'0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service,  <mask> by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist, currently  <mask> in Los Angeles.0-0-1-3 is an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base  <mask> on research by the National Institute on Alcohol Abuse and Alcoholism  <mask> binge  <mask> in college students.0.01'",
 "'is the debut studio album of H3llb3nt,  <mask> on February 20, 1996 by Fifth Colvmn Records.001 of 3 February 1997, which was  <mask> between the Government of the Republic of Rwanda, and FAPADER.003230 is a South Korean food manufacturer.0.04%Gas molecules in soil are in continuous thermal motion  <mask> to the kinetic theory of gasses, there is also collision between molecules - a random walk.0.04'",
 "'% of the votes were invalid.005.1999.06 is the fifth studio album by the South Korean singer and actress Uhm Jung-hwa.005 is a 1981 arcade game by Sega.

In [33]:
text_sentence = '0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service,  <mask> by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist, currently  <mask> in Los Angeles.0-0-1-3 is an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base  <mask> on research by the National Institute on Alcohol Abuse and Alcoholism  <mask> binge  <mask> in college students.0.01'

In [34]:
get_all_predictions(text_sentence)

0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service,  <mask> by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist, currently  <mask> in Los Angeles.0-0-1-3 is an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base  <mask> on research by the National Institute on Alcohol Abuse and Alcoholism  <mask> binge  <mask> in college students.0.01


{'bert': 'extends\ncorresponds\ntranslates\nis\nruns',
 'xlm': 'is\ncorrespond\npoints\namount\nruns'}

In [27]:
text_sentence = 'The stars <mask> towards each other'

In [28]:
get_all_predictions(text_sentence)

The stars <mask> towards each other


{'bert': 'pointed\nmove\nturn\npoint\nlook',
 'xlm': 'lean\nstar\nlook\nturn\npoint'}

In [ ]:
text_sentence = 'The <mask> gravitate towards each other'

In [ ]:
get_all_predictions(text_sentence)

The <mask> gravitate towards each other


{'bart': 'two\nstars\nthree\nway\nworld',
 'bert': 'two\nsexes\npair\nleaves\nbirds',
 'electra': 'two\nthree\npeople\nmen\nchildren',
 'roberta': 'characters\nstudents\ntwo\nplayers\nthree',
 'xlm': 'two\nanimals\npeople\nhumans\npair',
 'xlnet': 'a\nI\nthe\nand\nThe'}

In [ ]:
text_sentence = 'A hot <mask> will revive me'

In [ ]:
get_all_predictions(text_sentence)

A hot <mask> will revive me


{'bart': 'shower\ncup\nbath\ntodd\nchocolate',
 'bert': 'shower\nbath\nmeal\ndrink\nwater',
 'electra': 'summer\nspring\nsun\nday\nmoment',
 'roberta': 'bath\nshower\ndrink\nday\niron',
 'xlm': 'shower\nsummer\ndog\ngirl\nday',
 'xlnet': 'will\nwhich\nhot\nthat\nstar'}

In [ ]:
text_sentence = 'A hot soup will <mask> me'

In [ ]:
get_all_predictions(text_sentence)

A hot soup will <mask> me


{'bart': 'do\nalways\ncure\nnever\nnot',
 'bert': 'warm\nkill\nhelp\ncalm\nsave',
 'electra': 'give\nhelp\nsurprise\nsave\ntell',
 'roberta': 'calm\ncure\nrevive\nhelp\ncomfort',
 'xlm': 'kill\nlove\nhelp\nmake\nplease',
 'xlnet': 'me\ndo\nit\nto\nbe'}

In [ ]:
text_sentence = 'They <mask> their way to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They <mask> their way to the top of the mountain .


{'bart': 'then\nwere\nclimbed\nalso\nhad',
 'bert': 'made\nfound\npicked\nworked\nmake',
 'electra': 'made\nmake\nfound\nfind\nclawed',
 'roberta': 'made\nmake\nfound\nclimbed\nclimb',
 'xlm': 'made\nmake\nfound\nworked\nwork',
 'xlnet': 'their\nTheir\nYour\nmade\nHis'}

In [ ]:
text_sentence = 'They clawed their <mask> to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They clawed their <mask> to the top of the mountain .


{'bart': 'way\nown\nfeet\nhands\nfingers',
 'bert': 'way\nways\nfeet\nnails\nhands',
 'electra': 'way\nways\nhands\npath\nclaws',
 'roberta': 'way\nfeet\nfingers\nways\nhands',
 'xlm': 'feet\nway\narms\nhead\nback',
 'xlnet': 'way\nup\nupward\nback\nit'}

In [ ]:
text_sentence = 'He revived this <mask> of opera .'

In [ ]:
get_all_predictions(text_sentence)

He revived this <mask> of opera .


{'bart': 'work\nstyle\ntheme\ntradition\nopera',
 'bert': 'form\nstyle\ngenre\ntype\nkind',
 'electra': 'style\nform\nkind\ntype\ntradition',
 'roberta': 'form\nkind\ngenre\nstyle\ntype',
 'xlm': 'piece\nstyle\nopera\ntype\nsort',
 'xlnet': 'this\nrevival\ntradition\nform\ntype'}

In [ ]:
text_sentence = 'He <mask> this style of opera .'

In [ ]:
get_all_predictions(text_sentence)

He <mask> this style of opera .


{'bart': 'is\nwrote\nalso\nwas\nhas',
 'bert': 'pioneered\ndeveloped\ncreated\nintroduced\ninvented',
 'electra': 'likes\nloves\nloved\nliked\nchose',
 'roberta': 'liked\nprefers\npreferred\ncreated\nloved',
 'xlm': 'wrote\ncreated\nloved\nhad\nplayed',
 'xlnet': 'He\nThis\nShe\nhe\nIt'}

In [ ]:
text_sentence = 'They clawed their <mask> to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They clawed their <mask> to the top of the mountain .


{'bart': 'way\nown\nfeet\nhands\nfingers',
 'bert': 'way\nways\nfeet\nnails\nhands',
 'electra': 'way\nways\nhands\npath\nclaws',
 'roberta': 'way\nfeet\nfingers\nways\nhands',
 'xlm': 'feet\nway\narms\nhead\nback',
 'xlnet': 'way\nup\nupward\nback\nit'}

In [ ]:
text_sentence = 'They <mask> their way to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They <mask> their way to the top of the mountain .


{'bart': 'then\nwere\nclimbed\nalso\nhad',
 'bert': 'made\nfound\npicked\nworked\nmake',
 'electra': 'made\nmake\nfound\nfind\nclawed',
 'roberta': 'made\nmake\nfound\nclimbed\nclimb',
 'xlm': 'made\nmake\nfound\nworked\nwork',
 'xlnet': 'their\nTheir\nYour\nmade\nHis'}

In [ ]:
text_sentence = 'We rotate the <mask> so as to maximize the use of the soil .'

In [ ]:
get_all_predictions(text_sentence)

We rotate the <mask> so as to maximize the use of the soil .


{'bart': 'soil\nplant\nroot\nwheel\nplants',
 'bert': 'soil\nearth\nstones\nfields\nplants',
 'electra': 'water\nplants\nplates\nsystem\nbases',
 'roberta': 'soil\ncompost\ncrop\nplant\nplants',
 'xlm': 'plant\ngarden\ntree\nsurface\nfield',
 'xlnet': 'so\nin\nthe\nabout\nto'}

In [ ]:
text_sentence = 'We <mask> the crops so as to maximize the use of the soil .'

In [ ]:
get_all_predictions(text_sentence)

We <mask> the crops so as to maximize the use of the soil .


{'bart': 'can\nshould\nneed\nalso\nhave',
 'bert': 'grow\ntend\nplanted\ngrew\ncultivated',
 'electra': 'grow\nplant\nprepare\nbuild\nharvest',
 'roberta': 'rotate\nplanted\ngrow\nseeded\ncombine',
 'xlm': 'grow\nar\nplant\ncut\nincrease',
 'xlnet': 'the\nw\nWe\nwe\nThe'}

In [ ]:
text_sentence = "The <mask> don't harmonize ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> don't harmonize .


{'bart': 'two\nwords\nrules\nnumbers\nlyrics',
 'bert': 'words\nsongs\nvoices\ntwo\nsounds',
 'electra': 'two\nelements\nwords\nthings\nlines',
 'roberta': 'two\nlanguages\nrules\nwords\nthree',
 'xlm': 'two\nwords\ncolors\nlines\nlyrics',
 'xlnet': 'They\nthey\nThe\nthe\ne'}

In [ ]:
text_sentence = "The colors don't <mask> ."

In [ ]:
get_all_predictions(text_sentence)

The colors don't <mask> .


{'bart': 'match\nchange\nlook\nreally\nshow',
 'bert': 'change\nmatter\nmatch\nmove\nfade',
 'electra': 'change\nmatch\nwork\nmatter\nshow',
 'roberta': 'match\nchange\nmix\nmatter\nwork',
 'xlm': 'match\nchange\nwork\nmatter\nfit',
 'xlnet': 't\nnot\nand\ndo\nreally'}

In [ ]:
text_sentence = "The <mask> is being clogged by these operations ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> is being clogged by these operations .


{'bart': 'entire\nsystem\nwhole\nwater\nmain',
 'bert': 'river\nroad\nchannel\ncanal\nhighway',
 'electra': 'field\nsystem\nnetwork\nsite\ninternet',
 'roberta': 'system\npipeline\npipe\nriver\nnetwork',
 'xlm': 'world\nsystem\nplanet\nsite\ninternet',
 'xlnet': 'is\nIt\nm\nwas\nAm'}

In [ ]:
text_sentence = "The market is being <mask> by these operations ."

In [ ]:
get_all_predictions(text_sentence)

The market is being <mask> by these operations .


{'bart': 'served\nfurther\ndominated\ndriven\nsupported',
 'bert': 'served\ndominated\ndeveloped\nserviced\naffected',
 'electra': 'fed\ndriven\naffected\ndominated\ncontrolled',
 'roberta': 'served\ndominated\npenetrated\ndriven\ndisrupted',
 'xlm': 'affected\nfilled\nreached\ncreated\nused',
 'xlnet': 'being\ngoing\ning\ntaking\nwas'}

In [ ]:
text_sentence = "The <mask> was swelling with importance when she spoke of her son ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> was swelling with importance when she spoke of her son .


{'bart': 'mother\nwoman\nyoung\nformer\nactress',
 'bert': 'room\nwoman\nvoice\ndowager\nair',
 'electra': 'woman\nbody\nfamily\nheart\nworld',
 'roberta': 'room\nvoice\nair\nheart\ncongregation',
 'xlm': 'mother\nwoman\nwife\nmom\nMother',
 'xlnet': 'was\nIt\nI\nBody\nCenter'}

In [ ]:
text_sentence = "The mother was <mask> with importance when she spoke of her son ."

In [ ]:
get_all_predictions(text_sentence)

The mother was <mask> with importance when she spoke of her son .


{'bart': 'visibly\nalways\nalso\nnot\nsaid',
 'bert': 'filled\nspeaking\nstruck\nspoken\nheavy',
 'electra': 'filled\ntreated\nmet\npresented\ncharged',
 'roberta': 'speaking\nfilled\nglowing\nstruck\noverwhelmed',
 'xlm': 'treated\ntaken\nfilled\nimpressed\nconcerned',
 'xlnet': 'was\nof\nvery\nin\ngreat'}

In [ ]:
text_sentence = "Their <mask> inclines us to believe them ."

In [ ]:
get_all_predictions(text_sentence)

Their <mask> inclines us to believe them .


{'bart': 'own\nlack\nhistory\nsilence\narrogance',
 'bert': 'silence\nlogic\ntestimony\ntone\nlanguage',
 'electra': 'belief\npresence\nfaith\nvoice\ncourage',
 'roberta': 'logic\nsincerity\nfaith\nsilence\ntruth',
 'xlm': 'faith\ntruth\nstrength\npower\nfear',
 'xlnet': 'Their\nFaith\nGod\ntheir\nThey'}

In [ ]:
text_sentence = "Their language <mask> us to believe them ."

In [ ]:
get_all_predictions(text_sentence)

Their language <mask> us to believe them .


{'bart': 'is\nand\nmay\nhas\ntells',
 'bert': 'led\nleads\nconvinced\nforced\nencouraged',
 'electra': 'allows\ntells\nwants\nallowed\nrequires',
 'roberta': 'begs\nwants\ntells\nforces\ninvites',
 'xlm': 'allows\ntaught\nforces\ncauses\nwants',
 'xlnet': 'language\nis\nlanguages\nyou\nus'}

In [ ]:
text_sentence = "capture the <mask> of Spring   ."

In [ ]:
get_all_predictions(text_sentence)

capture the <mask> of Spring   .


{'bart': 'beauty\nspirit\nmoment\nmagic\nessence',
 'bert': 'light\nheat\nbreath\nrite\nscent',
 'electra': 'image\nlight\nscent\nblood\nshadow',
 'roberta': 'spirit\nbeauty\npromise\nRite\nSpirit',
 'xlm': 'beauty\nsky\nseason\ncolors\nflowers',
 'xlnet': 'of\nthe\nspirit\nday\nculture'}

In [ ]:
text_sentence = "<mask> the essence of Spring   ."

In [ ]:
get_all_predictions(text_sentence)

<mask> the essence of Spring   .


{'bart': 'Spring\n5\n4\n3\nS',
 'bert': 'for\nlike\nfeel\nfind\nand',
 'electra': 'in\nand\nconsider\nsee\nbring',
 'roberta': 'Understanding\nand\nAnd\n...\nFinding',
 'xlm': 'In\nand\nOn\nFor\nAbout',
 'xlnet': 'the\nelement\nmain\nessence\nspirit'}

In [ ]:
text_sentence = "He showered her with <mask> ."

In [ ]:
get_all_predictions(text_sentence)

He showered her with <mask> .


{'bart': 'kisses\ngifts\naffection\na\nlove',
 'bert': 'kisses\nsoap\nwarmth\nwater\nheat',
 'electra': 'water\nkisses\nflowers\ntea\ntears',
 'roberta': 'gifts\nflowers\nkisses\ncompliments\npraise',
 'xlm': 'sperm\nwater\ncum\nmilk\nwet',
 'xlnet': 'a\nwith\nby\nand\nthis'}

In [ ]:
text_sentence = "He <mask> her with presents ."

In [ ]:
get_all_predictions(text_sentence)

He <mask> her with presents .


{'bart': 'also\nthen\nwas\nwould\nalways',
 'bert': 'showered\npresented\nprovided\npresents\nleft',
 'electra': 'provided\npresented\nfilled\nshowered\noffered',
 'roberta': 'surprised\npresented\npresents\nspoiled\ngifted',
 'xlm': 'surprised\nfilled\nmarked\nbless\nleft',
 'xlnet': 'She\nHe\nHer\nWe\nIt'}

In [ ]:
text_sentence = "Am I supposed to swallow that <mask> ?"

In [ ]:
get_all_predictions(text_sentence)

Am I supposed to swallow that <mask> ?


{'bart': 'pill\nwhole\nshit\nand\none',
 'bert': 'pill\none\nshit\ndown\nthing',
 'electra': 'pill\nshit\nthing\nnow\none',
 'roberta': 'pill\nshit\ncrap\none\ncake',
 'xlm': 'liquid\nwater\npill\nthing\nmilk',
 'xlnet': 'that\nthis\na\nwhich\nthen'}

In [ ]:
text_sentence = "Am I supposed to <mask> that story ?"

In [ ]:
get_all_predictions(text_sentence)

Am I supposed to <mask> that story ?


{'bart': 'believe\nbe\nfeel\nbuy\nlike',
 'bert': 'believe\ntell\nwrite\nknow\nbe',
 'electra': 'tell\nwrite\nhave\nread\nknow',
 'roberta': 'tell\nbelieve\nwrite\nknow\nremember',
 'xlm': 'tell\nread\nwrite\nshare\ncontinue',
 'xlnet': 'write\ntell\nthat\nto\nbe'}

In [ ]:
text_sentence = "The <mask> glared down on us ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> glared down on us .


{'bart': 'sun\ndark\nold\nblack\ntwo',
 'bert': 'sun\nmoon\nguards\ndarkness\nsky',
 'electra': 'sun\nstars\ncamera\nclouds\nmen',
 'roberta': 'sun\ngods\nworld\nsky\nGods',
 'xlm': 'sun\nsky\nSun\nworld\nglass',
 'xlnet': 'The\nIt\nThey\nTo\nI'}

In [ ]:
text_sentence = "The sun <mask> down on us ."

In [ ]:
get_all_predictions(text_sentence)

The sun <mask> down on us .


{'bart': 'is\nwill\nwas\nhas\ncomes',
 'bert': 'beat\nshone\nbeats\ncame\nbore',
 'electra': 'came\nwas\nwent\nshone\nis',
 'roberta': 'beat\ncame\nbeats\nwent\nshone',
 'xlm': 'came\nwent\nis\nwas\nlooked',
 'xlnet': 'down\nsun\nup\nback\nbody'}

In [ ]:
text_sentence = "<mask> erupted in the country ."

In [ ]:
get_all_predictions(text_sentence)

<mask> erupted in the country .


{'bart': 'An\nThe\nAnother\nMore\nT',
 'bert': 'violence\nprotests\nriots\nchaos\nunrest',
 'electra': 'riots\nviolence\ngunfire\nexplosions\nfire',
 'roberta': 'Fighting\nWar\nviolence\nViolence\nPolitics',
 'xlm': 'which\nand\nthat\nIt\nit',
 'xlnet': 'erupted\nin\nexploded\nburst\nvolcano'}

In [ ]:
text_sentence = "Unrest <mask> in the country ."

In [ ]:
get_all_predictions(text_sentence)

Unrest <mask> in the country .


{'bart': 'continues\nis\nhas\nand\nin',
 'bert': 'erupted\ncontinued\noccurred\nbegan\nspread',
 'electra': 'continues\ncontinued\nremains\noccurred\nexists',
 'roberta': 's\ncontinues\nrising\npersists\ngrowing',
 'xlm': 'ling\nled\nlement\ns\nration',
 'xlnet': 'in\nhere\nis\nIn\nthere'}

In [ ]:
text_sentence = "poison someone 's <mask> ."

In [ ]:
get_all_predictions(text_sentence)

poison someone 's <mask> .


{'bart': 'water\ndrink\nbrain\nmouth\nblood',
 'bert': 'blood\nbody\nheart\nmind\nlife',
 'electra': 'body\nmouth\nmind\nskin\nface',
 'roberta': 'life\nname\nhead\nblood\nsoul',
 'xlm': 'life\nmind\nbody\nface\nhead',
 'xlnet': 's\nhouse\nplace\nhome\nbusiness'}

In [ ]:
text_sentence = "<mask> someone 's mind ."

In [ ]:
get_all_predictions(text_sentence)

<mask> someone 's mind .


{'bart': '3\n5\n4\nAn\nMake',
 'bert': 'in\nread\nreading\non\ninto',
 'electra': 'in\non\nchange\nnot\nand',
 'roberta': 'in\nIn\non\nReading\nRead',
 'xlm': 'in\nIn\non\nOn\nto',
 'xlnet': 'someone\ns\nin\nof\nto'}

In [ ]:
text_sentence = "<mask> were climbing after prices were lowered ."

In [ ]:
get_all_predictions(text_sentence)

<mask> were climbing after prices were lowered .


{'bart': 'C\nOil\nT\nPr\nU',
 'bert': 'prices\nrates\nsales\nwages\nrents',
 'electra': 'prices\nrates\nthey\ntemperatures\nstocks',
 'roberta': 'Sales\nThey\nand\nstocks\nShares',
 'xlm': 'prices\nThey\nSales\nWe\nThings',
 'xlnet': 'they\nwho\nclimbing\nThey\nthat'}

In [ ]:
text_sentence = "Sales were <mask> after prices were lowered ."

In [ ]:
get_all_predictions(text_sentence)

Sales were <mask> after prices were lowered .


{'bart': 'also\ndown\nup\nslightly\nflat',
 'bert': 'increased\nreduced\nboosted\nimproved\ndiscontinued',
 'electra': 'halted\nreduced\nclosed\nsuspended\nstopped',
 'roberta': 'steady\nup\nhigher\nflat\nstable',
 'xlm': 'increased\nclosed\nhigher\nstrong\nlower',
 'xlnet': 'until\nwere\nagain\ntoday\nwhere'}

In [ ]:
text_sentence = "This behavior will ruin your <mask> of winning the election ."

In [ ]:
get_all_predictions(text_sentence)

This behavior will ruin your <mask> of winning the election .


{'bart': 'chances\nchance\ncampaign\nparty\nhopes',
 'bert': 'chances\nchance\nprospects\nhope\nhopes',
 'electra': 'chances\nchance\nhopes\nhope\nrecord',
 'roberta': 'chances\nchance\nhopes\nprospects\nodds',
 'xlm': 'chances\nchance\nodds\ndream\nprospect',
 'xlnet': 'of\noff\nnot\nif\nfailing'}

In [ ]:
text_sentence = "This behavior will <mask> your chances of winning the election ."

In [ ]:
get_all_predictions(text_sentence)

This behavior will <mask> your chances of winning the election .


{'bart': 'not\nhelp\nincrease\ngreatly\nonly',
 'bert': 'increase\naffect\nreduce\nimprove\ndecrease',
 'electra': 'increase\nimprove\nreduce\nboost\nenhance',
 'roberta': 'affect\ndecrease\nhurt\nreduce\nincrease',
 'xlm': 'increase\nimprove\nreduce\ndecrease\nboost',
 'xlnet': 'your\naffect\nmy\nbe\ntheir'}

In [ ]:
text_sentence = "The sales tax is <mask> into the state income tax ."

In [ ]:
get_all_predictions(text_sentence)

The sales tax is <mask> into the state income tax .


{'bart': 'not\nalso\nthen\ncollected\nincorporated',
 'bert': 'incorporated\nconverted\nintegrated\nmerged\ncombined',
 'electra': 'converted\ndivided\nincorporated\nturned\ntranslated',
 'roberta': 'absorbed\nfolded\nrolled\nincorporated\nmerged',
 'xlm': 'included\nadded\npaid\nbuilt\nmade',
 'xlnet': 'into\nin\nout\nover\ngoing'}

In [ ]:
text_sentence = "The sales <mask> is absorbed into the state income tax ."

In [ ]:
get_all_predictions(text_sentence)

The sales <mask> is absorbed into the state income tax .


{'bart': 'tax\nand\ntaxes\nor\nof',
 'bert': 'tax\ntaxes\nrevenue\nincome\nlevy',
 'electra': 'tax\nrevenue\nvalue\nrate\namount',
 'roberta': 'tax\nrevenue\ntaxes\nfee\nprice',
 'xlm': 'tax\nprice\nincome\namount\ntaxes',
 'xlnet': 'tax\nin\nprogram\nof\nis'}

In [ ]:
text_sentence = "The invaders spread their <mask> all over the country ."

In [ ]:
get_all_predictions(text_sentence)

The invaders spread their <mask> all over the country .


{'bart': 'evil\narmy\nwicked\ndeadly\nwar',
 'bert': 'armies\narmy\npower\nattacks\nraids',
 'electra': 'weapons\nprey\nattacks\ntentacles\narms',
 'roberta': 'weapons\ndiseases\npropaganda\ntentacles\ndisease',
 'xlm': 'arms\npresence\nforces\npropaganda\nattack',
 'xlnet': 'all\nspread\nout\nway\nover'}

In [ ]:
text_sentence = "The invaders <mask> their language all over the country ."

In [ ]:
get_all_predictions(text_sentence)

The invaders <mask> their language all over the country .


{'bart': 'spread\nused\nleft\nwere\nhave',
 'bert': 'spread\nspoke\nused\nchanged\ntook',
 'electra': 'spread\nspeak\nspoken\nspoke\nread',
 'roberta': 'spread\nused\npracticed\nspoke\nintroduced',
 'xlm': 'spoke\nused\nspeak\nspread\nuse',
 'xlnet': 'have\nput\nof\nare\ntook'}

In [ ]:
text_sentence = "Interns have to <mask> for a few months ."

In [ ]:
get_all_predictions(text_sentence)

Interns have to <mask> for a few months .


{'bart': 'stay\nbe\nlive\nwork\nattend',
 'bert': 'stay\nwork\nwait\nleave\nlive',
 'electra': 'stay\nwait\nleave\nwork\nsleep',
 'roberta': 'stay\nleave\nwait\nwork\nreturn',
 'xlm': 'work\nwait\nstay\nleave\napply',
 'xlnet': 'go\nbe\nstay\nfor\nwait'}

In [ ]:
text_sentence = "They <mask> through the job candidates ."

In [ ]:
get_all_predictions(text_sentence)

They <mask> through the job candidates .


{'bart': 'are\nwill\nalso\nhave\nwere',
 'bert': 'went\nlooked\ngo\nwalked\ntalked',
 'electra': 'go\nwent\nget\nrun\ngot',
 'roberta': 'went\ngo\nrun\nran\nsort',
 'xlm': 'go\nwent\nwork\ncame\nwalk',
 'xlnet': 'went\nthrough\ngo\nway\nget'}

In [ ]:
text_sentence = "The dress <mask> her beautiful figure ."

In [ ]:
get_all_predictions(text_sentence)

The dress <mask> her beautiful figure .


{'bart': 'accent\nalso\nshowed\nperfectly\ncomple',
 'bert': 'hugged\nrevealed\nframed\nshowcased\noutlined',
 'electra': 'is\nwas\nfeatures\nshows\nfeatured',
 'roberta': 'hugged\nhighlighted\nemphasized\nhighlights\nreflected',
 'xlm': 'shows\nshowed\nof\nand\nmakes',
 'xlnet': 'her\nhis\nmy\nshe\nhe'}

In [ ]:
text_sentence = "<mask> the child with pride ."

In [ ]:
get_all_predictions(text_sentence)

<mask> the child with pride .


{'bart': '5\n4\n3\n10\n8',
 'bert': 'treat\ntreating\nfill\ntreated\nbless',
 'electra': 'bless\ntreat\nlove\nserve\npride',
 'roberta': 'adding\nImagine\nSee\nWatch\nsaid',
 'xlm': 'raise\ntreat\nhold\n1.\nLove',
 'xlnet': 'the\nchild\nof\nlittle\na'}

In [ ]:
text_sentence = "The listeners <mask> when he discussed his strange ideas ."

In [ ]:
get_all_predictions(text_sentence)

The listeners <mask> when he discussed his strange ideas .


{'bart': 'were\nlaughed\nalso\nloved\nhad',
 'bert': 'laughed\nlistened\nresponded\ngiggled\nsmiled',
 'electra': 'laughed\nagreed\ngasped\ncheered\nsmiled',
 'roberta': 'laughed\ncheered\ngasped\nlistened\napplauded',
 'xlm': 'laugh\nsurprised\nwatched\nenjoyed\nsmile',
 'xlnet': 'miss\near\nlisteners\nthat\nears'}

In [ ]:
text_sentence = "The path <mask> all the way to the top of the hill ."

In [ ]:
get_all_predictions(text_sentence)

The path <mask> all the way to the top of the hill .


{'bart': 'is\ncontinues\nfollows\nruns\nleads',
 'bert': 'led\nwent\nran\ncontinued\nstretched',
 'electra': 'goes\nruns\ncontinues\nleads\nis',
 'roberta': 'goes\ncontinues\nruns\nleads\nextends',
 'xlm': 'is\ngoes\nruns\ncontinues\ns',
 'xlnet': 'all\nup\nback\ngoing\npath'}

In [ ]:
text_sentence = "<mask> the contents of a book into a summary ."

In [ ]:
get_all_predictions(text_sentence)

<mask> the contents of a book into a summary .


{'bart': 'Summary\nAn\nBook\nThis\nThe',
 'bert': 'turn\nturning\nconvert\ntransform\ntranslate',
 'electra': 'combine\ntranslate\nturn\ntranslates\nput',
 'roberta': 'Putting\nPut\nBreak\nBreaking\nTurn',
 'xlm': 'Turn\nPut\nturn\nBreak\nChange',
 'xlnet': 'the\nsubject\nsummary\nin\nparts'}

##TroFi

In [ ]:
text_sentence = "Unfortunately , they wo n't work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . "

In [ ]:
get_all_predictions(text_sentence)

Unfortunately , they wo n't work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . 


{'bart': 'stuck\nlost\na\ninto\nhit',
 'bert': 'stuck\nlost\nstranded\noff\ncaught',
 'electra': 'off\nout\nlost\naway\nup',
 'roberta': 'stuck\nlost\nstranded\nhit\ncaught',
 'xlm': 'stuck\nlost\ncaught\nhit\nseen',
 'xlnet': 'lost\nget\nstuck\ndrowned\nout'}

In [ ]:
text_sentence = "It also vindicates Pratt 's efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co"

In [ ]:
get_all_predictions(text_sentence)

It also vindicates Pratt 's efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co


{'bart': 'repair\nrebuild\nimprove\nfix\novercome',
 'bert': 'address\nresolve\nsolve\nfix\ncorrect',
 'electra': 'solve\naddress\novercome\nfix\nresolve',
 'roberta': 'fix\naddress\ncorrect\nresolve\nremedy',
 'xlm': 'address\nsolve\nresolve\nfix\ncorrect',
 'xlnet': 'solve\nto\naddress\nfix\nthe'}

In [ ]:
text_sentence = "Weeks later , '' Penthouse finally realized that Dominion was intentionally <mask> the deal , '' sought other financing , failed , and had to abandon the project , the judge said "

In [ ]:
get_all_predictions(text_sentence)

Weeks later , '' Penthouse finally realized that Dominion was intentionally <mask> the deal , '' sought other financing , failed , and had to abandon the project , the judge said 


{'bart': 'trying\nmisleading\nlying\nwithholding\nhiding',
 'bert': 'violating\ndelaying\nbreaking\nblowing\nblocking',
 'electra': 'blocking\nbreaking\nabandoning\nruining\nviolating',
 'roberta': 'rigging\nbreaking\nkilling\nmanipulating\ncutting',
 'xlm': 'breaking\nmaking\nin\nselling\ntaking',
 'xlnet': 'in\nbehind\ninto\non\nfinancing'}

In [ ]:
text_sentence = "After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings "

In [ ]:
get_all_predictions(text_sentence)

After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings 


{'bart': 'find\nlook\ntake\nkeep\nget',
 'bert': 'seek\nmake\naccept\nattend\npursue',
 'electra': 'find\nmake\nget\ncreate\nreach',
 'roberta': 'fill\nfind\nseek\ncreate\nmake',
 'xlm': 'fill\nopen\nfind\npursue\nsecure',
 'xlnet': 'to\njob\nmake\nopen\naccess'}

In [ ]:
text_sentence = "She 's been able to <mask> to this policy , largely because of the nature of her music "

In [ ]:
get_all_predictions(text_sentence)

She 's been able to <mask> to this policy , largely because of the nature of her music 


{'bart': 'maintain\nremain\navoid\nhold\ncontinue',
 'bert': 'adhere\nstick\nagree\nadapt\nconform',
 'electra': 'adhere\nstick\nadapt\nconform\nagree',
 'roberta': 'adhere\nstick\nadapt\nconform\nappeal',
 'xlm': 'stick\ncontribute\nagree\nconform\nadapt',
 'xlnet': 'come\nget\nto\ngo\naccess'}

In [ ]:
text_sentence = "The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated "

In [ ]:
get_all_predictions(text_sentence)

The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated 


{'bart': 'extend\nmeet\nchange\npostpone\ndelay',
 'bert': 'meet\naccept\nextend\nset\nreach',
 'electra': 'meet\nreach\nset\nmiss\npass',
 'roberta': 'meet\nmiss\nextend\ndelay\naccept',
 'xlm': 'meet\nreach\nmiss\nhit\nachieve',
 'xlnet': 'meet\nignore\naccept\nto\navoid'}

In [ ]:
text_sentence = "'' I found myself <mask> out on the steps in boot camp , '' he recalls "

In [ ]:
get_all_predictions(text_sentence)

'' I found myself <mask> out on the steps in boot camp , '' he recalls 


{'bart': 'in\ncrying\ngetting\nwith\non',
 'bert': 'passed\nhiding\nhanging\nbleeding\nsprawled',
 'electra': 'hanging\ngoing\nwalking\ncamping\nlaying',
 'roberta': 'passed\nhanging\ncrying\nsitting\ncamping',
 'xlm': 'walking\nrunning\nstanding\ngoing\nmissing',
 'xlnet': 'yourself\nway\nout\nback\nmyself'}

In [ ]:
text_sentence = "Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger "

In [ ]:
get_all_predictions(text_sentence)

Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger 


{'bart': 'who\nkilled\nof\nand\ninvolved',
 'bert': 'involved\naboard\nparticipating\nkilled\nlost',
 'electra': 'involved\nworking\nliving\nremaining\noperating',
 'roberta': 'killed\ninvolved\ninjured\nlost\nmissing',
 'xlm': 'involved\nkilled\noperating\nworking\npresent',
 'xlnet': 'involved\nin\nkilled\nwho\ninjured'}

In [ ]:
text_sentence = "Incumbent Spyros Kyprianou was <mask> out in voting last week"

In [ ]:
get_all_predictions(text_sentence)

Incumbent Spyros Kyprianou was <mask> out in voting last week


{'bart': 'the\nnarrowly\nalso\ndeclared\nforced',
 'bert': 'knocked\nvoted\nleft\nedged\nstill',
 'electra': 'knocked\nruled\nwiped\nshut\nnot',
 'roberta': 'shut\nedged\nknocked\nwiped\nvoted',
 'xlm': 'missing\nnot\nall\nmissed\nsold',
 'xlnet': 'was\nout\nalso\nnot\nhimself'}